# Feature transformation

Used to test the impact of various feature transformations on the dataset, and ultimately outputs a dataset with those transformations applied on it.

## Dataset loading

In [1]:
import pandas as pd
import python_proj.utils.exp_utils as exp_utils

input_data_file_name = "dataset_subsampled"

data_path_format = exp_utils.BASE_PATH + "/final_data/{data_file_name}.csv"
data_path = data_path_format.format(data_file_name=input_data_file_name)
print(f"{data_path=}")

meta_header_count = 5

df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)

print(f"{len(df)=}.")
print(f"{len(df.columns)=}\n")

df.describe()

data_path='/workspaces/msc_thesis/data//final_data/dataset_subsampled.csv'
len(df)=1216221.
len(df.columns)=51



,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,ControlIntraProjectPullRequestExperienceOfIntegrator,IntraProjectSubmitterPullRequestSubmissionCount,IntraProjectSubmitterPullRequestSuccessRate,IntraProjectSubmitterPullRequestCommentCount,IntraProjectSharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator,...,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount,WeightedEcosystemSecondOrderInDegreeCentrality,WeightedEcosystemSecondOrderOutDegreeCentrality,WeightedIntraProjectSecondOrderInDegreeCentrality,WeightedIntraProjectSecondOrderOutDegreeCentrality,EcosystemIntegratorToSubmitterLinkIntensity,EcosystemSubmitterToIntegratorLinkIntensity,IntraProjectIntegratorToSubmitterLinkIntensity,IntraProjectSubmitterToIntegratorLinkIntensity
count,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,...,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06
mean,1.473143e+08,5.093466e+06,8.082764e+02,3.685136e+04,4.194136e+00,4.019659e+01,9.860225e+00,5.027950e-01,1.018910e+01,7.866630e+00,...,2.574277e-01,1.805102e+00,1.583814e+01,1.333045e+01,3.120385e+01,3.110127e+01,1.676589e+00,1.739200e+00,1.122238e+00,1.352802e+00
std,9.953631e+07,7.967840e+06,2.296849e+03,1.538714e+05,5.357660e+01,1.164114e+02,3.460383e+01,4.710790e-01,3.297641e+01,3.319649e+01,...,2.923879e+00,1.901014e+01,6.758305e+02,6.055197e+02,1.377436e+03,1.938964e+03,8.313866e+00,8.337128e+00,4.751919e+00,4.800951e+00
min,6.870000e+02,1.000000e+00,1.000000e+00,1.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.925868e+07,4.673300e+05,5.000000e+01,3.696667e+01,1.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.367237e+08,1.634092e+06,1.990000e+02,8.971167e+02,1.000000e+00,1.100000e+01,1.000000e+00,6.666667e-01,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,5.782397e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.260987e+08,6.254382e+06,6.840000e+02,8.001133e+03,3.000000e+00,3.800000e+01,9.000000e+00,1.000000e+00,7.000000e+00,6.000000e+00,...,0.000000e+00,0.000000e+00,1.461533e+00,8.202236e-01,1.275396e+00,9.194294e-01,2.434832e-01,4.730551e-01,4.800107e-01,9.461103e-01
max,3.617734e+08,5.973444e+07,8.297600e+04,4.489490e+06,1.000000e+04,4.274000e+03,1.703000e+03,1.000000e+00,1.984000e+03,1.703000e+03,...,7.410000e+02,2.764000e+03,2.811220e+05,2.723745e+05,1.035309e+06,1.488838e+06,1.787202e+03,1.787202e+03,4.319132e+02,4.319132e+02


In [2]:
predictors = df.columns[meta_header_count:]
numeric_predictors = df[predictors].select_dtypes(include="number")

## Log-transformation

Applies one-off log-transform on count predictors (PR submission count, etc.).

In [3]:
import numpy as np
import regex as re

df_transformed = df.copy()

sr_re = r".*SuccessRate.*"

count_predictors = [
    field for field in numeric_predictors.columns if not re.match(sr_re, field)
]

print(f"{count_predictors=}")

count_predictors=['ControlPullRequestLifeTimeInMinutes', 'ControlNumberOfCommitsInPullRequest', 'ControlIntraProjectPullRequestExperienceOfIntegrator', 'IntraProjectSubmitterPullRequestSubmissionCount', 'IntraProjectSubmitterPullRequestCommentCount', 'IntraProjectSharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator', 'IntraProjectSharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter', 'EcosystemSharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter', 'EcosystemSharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter.1', 'EcosystemExperienceSubmitterPullRequestSubmissionCount', 'EcosystemExperienceSubmitterPullRequestCommentCount', 'DependencyEcosystemExperienceSubmitterPullRequestSubmissionCount', 'DependencyEcosystemExperienceSubmitterPullRequestCommentCount', 'NonDependencyEcosystemExperienceSubmitterPullRequestSubmissionCount', 'NonDependencyEcosystemExperienceSubmitterPullRequestCommentCount', 'InversedDependencyEcosyste

In [4]:
# Applies one-off log-transform
for field in count_predictors:
    new_key = f"ln(1 + {field})"
    df_transformed[new_key] = df_transformed[field].apply(lambda x: np.log(1 + x))
    df_transformed = df_transformed.drop(field, axis=1)

df_transformed.describe()

,ID,Submitter ID,PR Number,IntraProjectSubmitterPullRequestSuccessRate,EcosystemExperienceSubmitterPullRequestSuccessRate,DependencyEcosystemExperienceSubmitterPullRequestSuccessRate,NonDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,ln(1 + ControlPullRequestLifeTimeInMinutes),ln(1 + ControlNumberOfCommitsInPullRequest),...,ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount),ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueCommentCount),ln(1 + WeightedEcosystemSecondOrderInDegreeCentrality),ln(1 + WeightedEcosystemSecondOrderOutDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderInDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderOutDegreeCentrality),ln(1 + EcosystemIntegratorToSubmitterLinkIntensity),ln(1 + EcosystemSubmitterToIntegratorLinkIntensity),ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity),ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity)
count,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,...,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06
mean,1.473143e+08,5.093466e+06,8.082764e+02,5.027950e-01,4.028468e-01,9.859457e-02,3.751558e-01,5.921727e-02,6.436513e+00,1.090798e+00,...,6.303926e-02,1.389330e-01,6.504374e-01,5.236796e-01,6.751089e-01,6.005097e-01,3.536191e-01,3.765622e-01,3.416321e-01,4.354887e-01
std,9.953631e+07,7.967840e+06,2.296849e+03,4.710790e-01,4.523912e-01,2.907958e-01,4.479800e-01,2.284907e-01,3.453285e+00,6.587747e-01,...,3.476386e-01,6.535100e-01,1.104536e+00,1.030077e+00,1.250821e+00,1.188442e+00,7.703750e-01,7.813927e-01,6.761163e-01,7.130512e-01
min,6.870000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.652930e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.925868e+07,4.673300e+05,5.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.636709e+00,6.931472e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.367237e+08,1.634092e+06,1.990000e+02,6.666667e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.800300e+00,6.931472e-01,...,0.000000e+00,0.000000e+00,5.621394e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.260987e+08,6.254382e+06,6.840000e+02,1.000000e+00,9.393939e-01,0.000000e+00,9.230769e-01,0.000000e+00,8.987463e+00,1.386294e+00,...,0.000000e+00,0.000000e+00,9.007845e-01,5.989593e-01,8.221541e-01,6.520280e-01,2.179164e-01,3.873386e-01,3.920493e-01,6.658326e-01
max,3.617734e+08,5.973444e+07,8.297600e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.531725e+01,9.210440e+00,...,6.609349e+00,7.924796e+00,1.254655e+01,1.251494e+01,1.385021e+01,1.421351e+01,7.488966e+00,7.488966e+00,6.070537e+00,6.070537e+00


In [5]:
# Collects the new numeric fields for the transformed dataframe.
transformed_predictors = df_transformed.columns[meta_header_count:]
transformed_numeric_predictors = df_transformed[transformed_predictors].select_dtypes(
    include="number"
)

## Feature scaling

Applies min-max scaling on the features.
This is applied to the transformed and the untransformed data.

In [6]:
def scale(_df: pd.DataFrame, scaled_fields: pd.Series):
    scaled_df = _df.copy()

    for feature in scaled_fields:
        feature_min = scaled_df[feature].min()
        feature_max = scaled_df[feature].max()
        feature_delta = feature_max - feature_min

        scaled_df[feature] = (
            scaled_df[feature].subtract(feature_min).divide(feature_delta)
        )

    return scaled_df

In [7]:
df_transformed = scale(df_transformed, transformed_numeric_predictors.columns)
df_transformed.describe()

,ID,Submitter ID,PR Number,IntraProjectSubmitterPullRequestSuccessRate,EcosystemExperienceSubmitterPullRequestSuccessRate,DependencyEcosystemExperienceSubmitterPullRequestSuccessRate,NonDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,ln(1 + ControlPullRequestLifeTimeInMinutes),ln(1 + ControlNumberOfCommitsInPullRequest),...,ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount),ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueCommentCount),ln(1 + WeightedEcosystemSecondOrderInDegreeCentrality),ln(1 + WeightedEcosystemSecondOrderOutDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderInDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderOutDegreeCentrality),ln(1 + EcosystemIntegratorToSubmitterLinkIntensity),ln(1 + EcosystemSubmitterToIntegratorLinkIntensity),ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity),ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity)
count,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,...,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06
mean,1.473143e+08,5.093466e+06,8.082764e+02,5.027950e-01,4.028468e-01,9.859457e-02,3.751558e-01,5.921727e-02,4.195870e-01,1.184307e-01,...,9.537892e-03,1.753143e-02,5.184194e-02,4.184437e-02,4.874358e-02,4.224923e-02,4.721868e-02,5.028227e-02,5.627708e-02,7.173808e-02
std,9.953631e+07,7.967840e+06,2.296849e+03,4.710790e-01,4.523912e-01,2.907958e-01,4.479800e-01,2.284907e-01,2.256942e-01,7.152478e-02,...,5.259800e-02,8.246395e-02,8.803506e-02,8.230781e-02,9.031063e-02,8.361355e-02,1.028680e-01,1.043392e-01,1.113767e-01,1.174610e-01
min,6.870000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.925868e+07,4.673300e+05,5.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.366019e-01,7.525668e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.367237e+08,1.634092e+06,1.990000e+02,6.666667e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.433628e-01,7.525668e-02,...,0.000000e+00,0.000000e+00,4.480431e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.260987e+08,6.254382e+06,6.840000e+02,1.000000e+00,9.393939e-01,0.000000e+00,9.230769e-01,0.000000e+00,5.863080e-01,1.505134e-01,...,0.000000e+00,0.000000e+00,7.179541e-02,4.785956e-02,5.936040e-02,4.587383e-02,2.909834e-02,5.172123e-02,6.458231e-02,1.096827e-01
max,3.617734e+08,5.973444e+07,8.297600e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [8]:
df = scale(df, numeric_predictors.columns)
df.describe()

,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,ControlIntraProjectPullRequestExperienceOfIntegrator,IntraProjectSubmitterPullRequestSubmissionCount,IntraProjectSubmitterPullRequestSuccessRate,IntraProjectSubmitterPullRequestCommentCount,IntraProjectSharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator,...,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount,WeightedEcosystemSecondOrderInDegreeCentrality,WeightedEcosystemSecondOrderOutDegreeCentrality,WeightedIntraProjectSecondOrderInDegreeCentrality,WeightedIntraProjectSecondOrderOutDegreeCentrality,EcosystemIntegratorToSubmitterLinkIntensity,EcosystemSubmitterToIntegratorLinkIntensity,IntraProjectIntegratorToSubmitterLinkIntensity,IntraProjectSubmitterToIntegratorLinkIntensity
count,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,...,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06
mean,1.473143e+08,5.093466e+06,8.082764e+02,8.208360e-03,4.194136e-04,9.404911e-03,5.789915e-03,5.027950e-01,5.135633e-03,4.619278e-03,...,3.474058e-04,6.530760e-04,5.633901e-05,4.894162e-05,3.013965e-05,2.088962e-05,9.381081e-04,9.731412e-04,2.598294e-03,3.132114e-03
std,9.953631e+07,7.967840e+06,2.296849e+03,3.427369e-02,5.357660e-03,2.723710e-02,2.031933e-02,4.710790e-01,1.662117e-02,1.949295e-02,...,3.945856e-03,6.877763e-03,2.404047e-03,2.223115e-03,1.330459e-03,1.302334e-03,4.651889e-03,4.664904e-03,1.100202e-02,1.111554e-02
min,6.870000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.925868e+07,4.673300e+05,5.000000e+01,8.230334e-06,1.000000e-04,7.019186e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.367237e+08,1.634092e+06,1.990000e+02,1.998223e-04,1.000000e-04,2.573701e-03,5.871991e-04,6.666667e-01,0.000000e+00,5.871991e-04,...,0.000000e+00,0.000000e+00,2.056899e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.260987e+08,6.254382e+06,6.840000e+02,1.782188e-03,3.000000e-04,8.890969e-03,5.284792e-03,1.000000e+00,3.528226e-03,3.523194e-03,...,0.000000e+00,0.000000e+00,5.198929e-06,3.011382e-06,1.231899e-06,6.175482e-07,1.362370e-04,2.646903e-04,1.111359e-03,2.190510e-03
max,3.617734e+08,5.973444e+07,8.297600e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


## Dataset safe to file

In [9]:
output_data_file_name = "dataset_transformed"
output_path = data_path_format.format(data_file_name=output_data_file_name)
print(f"Outputting to '{output_path}'")

df_transformed.to_csv(output_path, index=False)

df_transformed.describe()

Outputting to '/workspaces/msc_thesis/data//final_data/dataset_transformed.csv'


,ID,Submitter ID,PR Number,IntraProjectSubmitterPullRequestSuccessRate,EcosystemExperienceSubmitterPullRequestSuccessRate,DependencyEcosystemExperienceSubmitterPullRequestSuccessRate,NonDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,ln(1 + ControlPullRequestLifeTimeInMinutes),ln(1 + ControlNumberOfCommitsInPullRequest),...,ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount),ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueCommentCount),ln(1 + WeightedEcosystemSecondOrderInDegreeCentrality),ln(1 + WeightedEcosystemSecondOrderOutDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderInDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderOutDegreeCentrality),ln(1 + EcosystemIntegratorToSubmitterLinkIntensity),ln(1 + EcosystemSubmitterToIntegratorLinkIntensity),ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity),ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity)
count,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,...,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06
mean,1.473143e+08,5.093466e+06,8.082764e+02,5.027950e-01,4.028468e-01,9.859457e-02,3.751558e-01,5.921727e-02,4.195870e-01,1.184307e-01,...,9.537892e-03,1.753143e-02,5.184194e-02,4.184437e-02,4.874358e-02,4.224923e-02,4.721868e-02,5.028227e-02,5.627708e-02,7.173808e-02
std,9.953631e+07,7.967840e+06,2.296849e+03,4.710790e-01,4.523912e-01,2.907958e-01,4.479800e-01,2.284907e-01,2.256942e-01,7.152478e-02,...,5.259800e-02,8.246395e-02,8.803506e-02,8.230781e-02,9.031063e-02,8.361355e-02,1.028680e-01,1.043392e-01,1.113767e-01,1.174610e-01
min,6.870000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.925868e+07,4.673300e+05,5.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.366019e-01,7.525668e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.367237e+08,1.634092e+06,1.990000e+02,6.666667e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.433628e-01,7.525668e-02,...,0.000000e+00,0.000000e+00,4.480431e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.260987e+08,6.254382e+06,6.840000e+02,1.000000e+00,9.393939e-01,0.000000e+00,9.230769e-01,0.000000e+00,5.863080e-01,1.505134e-01,...,0.000000e+00,0.000000e+00,7.179541e-02,4.785956e-02,5.936040e-02,4.587383e-02,2.909834e-02,5.172123e-02,6.458231e-02,1.096827e-01
max,3.617734e+08,5.973444e+07,8.297600e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [10]:
output_data_file_name = "dataset_untransformed"
output_path = data_path_format.format(data_file_name=output_data_file_name)
print(f"Outputting to '{output_path}'")

df.to_csv(output_path, index=False)

df.describe()

Outputting to '/workspaces/msc_thesis/data//final_data/dataset_untransformed.csv'


,ID,Submitter ID,PR Number,ControlPullRequestLifeTimeInMinutes,ControlNumberOfCommitsInPullRequest,ControlIntraProjectPullRequestExperienceOfIntegrator,IntraProjectSubmitterPullRequestSubmissionCount,IntraProjectSubmitterPullRequestSuccessRate,IntraProjectSubmitterPullRequestCommentCount,IntraProjectSharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator,...,InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount,InversedDependencyEcosystemExperienceSubmitterIssueCommentCount,WeightedEcosystemSecondOrderInDegreeCentrality,WeightedEcosystemSecondOrderOutDegreeCentrality,WeightedIntraProjectSecondOrderInDegreeCentrality,WeightedIntraProjectSecondOrderOutDegreeCentrality,EcosystemIntegratorToSubmitterLinkIntensity,EcosystemSubmitterToIntegratorLinkIntensity,IntraProjectIntegratorToSubmitterLinkIntensity,IntraProjectSubmitterToIntegratorLinkIntensity
count,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,...,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06
mean,1.473143e+08,5.093466e+06,8.082764e+02,8.208360e-03,4.194136e-04,9.404911e-03,5.789915e-03,5.027950e-01,5.135633e-03,4.619278e-03,...,3.474058e-04,6.530760e-04,5.633901e-05,4.894162e-05,3.013965e-05,2.088962e-05,9.381081e-04,9.731412e-04,2.598294e-03,3.132114e-03
std,9.953631e+07,7.967840e+06,2.296849e+03,3.427369e-02,5.357660e-03,2.723710e-02,2.031933e-02,4.710790e-01,1.662117e-02,1.949295e-02,...,3.945856e-03,6.877763e-03,2.404047e-03,2.223115e-03,1.330459e-03,1.302334e-03,4.651889e-03,4.664904e-03,1.100202e-02,1.111554e-02
min,6.870000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.925868e+07,4.673300e+05,5.000000e+01,8.230334e-06,1.000000e-04,7.019186e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.367237e+08,1.634092e+06,1.990000e+02,1.998223e-04,1.000000e-04,2.573701e-03,5.871991e-04,6.666667e-01,0.000000e+00,5.871991e-04,...,0.000000e+00,0.000000e+00,2.056899e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.260987e+08,6.254382e+06,6.840000e+02,1.782188e-03,3.000000e-04,8.890969e-03,5.284792e-03,1.000000e+00,3.528226e-03,3.523194e-03,...,0.000000e+00,0.000000e+00,5.198929e-06,3.011382e-06,1.231899e-06,6.175482e-07,1.362370e-04,2.646903e-04,1.111359e-03,2.190510e-03
max,3.617734e+08,5.973444e+07,8.297600e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [11]:
import datetime

print(datetime.datetime.now())

2024-02-16 14:47:34.213568
